In [1]:
import pandas as pd

In [5]:
data = pd.read_csv('data/churn.csv')
data_test = pd.read_csv('data/churn_test.csv')
data_train = pd.read_csv('data/churn_train.csv')

In [25]:
data.T

,0,1,2,3,4,5,6,7,8,9,...,49990,49991,49992,49993,49994,49995,49996,49997,49998,49999
avg_dist,3.67,8.26,0.77,2.36,3.13,10.56,3.95,2.04,4.36,2.37,...,3.38,1.06,7.58,2.53,2.25,5.63,0,3.86,4.58,3.49
avg_rating_by_driver,5,5,5,4.9,4.9,5,4,5,5,5,...,5,5,5,4.7,4.5,4.2,4,5,3.5,5
avg_rating_of_driver,4.7,5,4.3,4.6,4.4,3.5,NaN,5,4.5,NaN,...,4.7,5,1,4.8,4.6,5,NaN,5,3,NaN
avg_surge,1.1,1,1,1.14,1.19,1,1,1,1,1,...,1.08,1.25,1,1.11,1.44,1,1,1,1,1
city,King's Landing,Astapor,Astapor,King's Landing,Winterfell,Winterfell,Astapor,Winterfell,Winterfell,Winterfell,...,Astapor,Winterfell,King's Landing,Astapor,Astapor,King's Landing,Astapor,Winterfell,Astapor,Astapor
last_trip_date,2014-06-17,2014-05-05,2014-01-07,2014-06-29,2014-03-15,2014-06-06,2014-01-25,2014-01-29,2014-02-01,2014-01-05,...,2014-05-18,2014-06-29,2014-01-19,2014-07-01,2014-05-31,2014-06-05,2014-01-25,2014-05-22,2014-01-15,2014-04-20
phone,iPhone,Android,iPhone,iPhone,Android,iPhone,Android,iPhone,Android,Android,...,iPhone,iPhone,iPhone,iPhone,iPhone,iPhone,iPhone,Android,iPhone,Android
signup_date,2014-01-25,2014-01-29,2014-01-06,2014-01-10,2014-01-27,2014-01-09,2014-01-24,2014-01-28,2014-01-21,2014-01-03,...,2014-01-13,2014-01-08,2014-01-18,2014-01-03,2014-01-03,2014-01-25,2014-01-24,2014-01-31,2014-01-14,2014-01-18
surge_pct,15.4,0,0,20,11.8,0,0,0,0,0,...,33.3,100,0,11.1,37.5,0,0,0,0,0
trips_in_first_30_days,4,0,3,9,14,2,1,2,2,1,...,1,0,1,3,1,0,1,0,2,0


# Data cleaning pipeline:

- date to datetime
- remove NA


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   avg_dist                50000 non-null  float64
 1   avg_rating_by_driver    49799 non-null  float64
 2   avg_rating_of_driver    41878 non-null  float64
 3   avg_surge               50000 non-null  float64
 4   city                    50000 non-null  object 
 5   last_trip_date          50000 non-null  object 
 6   phone                   49604 non-null  object 
 7   signup_date             50000 non-null  object 
 8   surge_pct               50000 non-null  float64
 9   trips_in_first_30_days  50000 non-null  int64  
 10  luxury_car_user         50000 non-null  int64  
 11  weekday_pct             50000 non-null  float64
 12  last_trip               50000 non-null  object 
 13  churn                   50000 non-null  int64  
dtypes: float64(6), int64(3), object(5)
mem

In [26]:
date = '06-01-2014'
data['last_trip'] = pd.to_datetime(data.last_trip_date)
data['last_trip'] = data['last_trip'].dt.date
data['churn'] = (data.last_trip < date)*1
data['signup_date'] = pd.to_datetime(data.signup_date)
data['signup_date'] = data['signup_date'].dt.date
data['luxury_car_user'] = data.luxury_car_user*1

In [30]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41445 entries, 0 to 49998
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   avg_dist                41445 non-null  float64
 1   avg_rating_by_driver    41445 non-null  float64
 2   avg_rating_of_driver    41445 non-null  float64
 3   avg_surge               41445 non-null  float64
 4   city                    41445 non-null  object 
 5   last_trip_date          41445 non-null  object 
 6   phone                   41445 non-null  object 
 7   signup_date             41445 non-null  object 
 8   surge_pct               41445 non-null  float64
 9   trips_in_first_30_days  41445 non-null  int64  
 10  luxury_car_user         41445 non-null  int64  
 11  weekday_pct             41445 non-null  float64
 12  last_trip               41445 non-null  object 
 13  churn                   41445 non-null  int64  
dtypes: float64(6), int64(3), object(5)
mem

In [31]:
data.city.value_counts()

Winterfell        19161
Astapor           13509
King's Landing     8775
Name: city, dtype: int64

In [33]:
data.phone.value_counts()

iPhone     29352
Android    12093
Name: phone, dtype: int64

In [32]:
data.churn.value_counts()

1    24410
0    17035
Name: churn, dtype: int64

In [40]:
X = data[['avg_dist', 
     'avg_rating_by_driver', 
     'avg_rating_of_driver', 
     'avg_surge', 'surge_pct', 
     'trips_in_first_30_days', 
     'weekday_pct']].values
y = data.churn.values